In [0]:
print("Healthy")

In [0]:
import pandas as pd 
import numpy as np 
import duckdb
from re import search
from sqlalchemy import create_engine

import warnings
warnings.filterwarnings('ignore')


from datetime import datetime, timedelta
d = datetime.today() - timedelta(days = 1)
d = d.strftime('%Y-%m-%d')

date = "'" + d + "'"
print(date, d)

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email import encoders

import mysql.connector
import json
with open('/Workspace/Credentials/db_data.json', 'r') as fp:
    data = json.load(fp)

Dumpdb = mysql.connector.connect(
  host=data['Dumpdb']['host'],
  user=data['Dumpdb']['user'],
  passwd=data['Dumpdb']['passwd'],
  database = data['Dumpdb']['database']
)


Mifosdb = mysql.connector.connect(
  host=data['mifos']['host'],
  user=data['mifos']['user'],
  passwd=data['mifos']['passwd'],
  database = data['mifos']['database']
)


vbizdb = mysql.connector.connect(
  host=data['vbizdb']['host'],
  user=data['vbizdb']['user'],
  passwd=data['vbizdb']['passwd'],
  database = data['vbizdb']['database']
)

host = data['redshift']['host']
user = data['redshift']['user']
passwd = data['redshift']['passwd']
database = data['redshift']['database']

conn = create_engine(f"postgresql+psycopg2://{user}:{passwd}@{host}:5439/{database}")


pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)


In [0]:
# Load in credentials
with open('/Workspace/Credentials/db_data.json', 'r') as fp:
    data = json.load(fp)

# MIFOS_DB
mifos_driver = "org.mariadb.jdbc.Driver"
mifos_database_host = data['mifos']['host']
mifos_database_port = "3306" # update if you use a non-default port
mifos_database_name = data['mifos']['database']
mifos_user=data['mifos']['user']
mifos_password = data['mifos']['passwd']
mifos_url = f"jdbc:mysql://{mifos_database_host}:{mifos_database_port}/{mifos_database_name}"

# REDSHIFT_DWH
redshift_driver = "org.postgresql.Driver"
redshift_host = data['redshift']['host']
redshift_db_name = data['redshift']['database']
redshift_user = data['redshift']['user']
redshift_password = data['redshift']['passwd']
redshift_url = f"jdbc:postgresql://{redshift_host}:5439/{redshift_db_name}"


# redshift connection for dask
redshift_conn = f'postgresql://{redshift_user}:{redshift_password}@{redshift_host}:5439/{redshift_db_name}'

In [0]:
def load_table(schema_name, table_name, dataframe):
    engine = create_engine(f'postgresql+psycopg2://{redshift_user}:{redshift_password}@{redshift_host}:5439/{redshift_db_name}')
    
    sql = f'''
    CALL truncate_if_exists('{schema_name}', '{table_name}');
    '''

    with engine.begin() as con:     
        con.execute(sql)    

    chunk_size = 10000
    num_rows = len(dataframe)

    conn = create_engine(f'postgresql+psycopg2://{redshift_user}:{redshift_password}@{redshift_host}:5439/{redshift_db_name}')
    num_iterations = (num_rows // chunk_size) + 1 if num_rows % chunk_size != 0 else num_rows // chunk_size

    for i in range(num_iterations):
        start_idx = i * chunk_size
        end_idx = (i + 1) * chunk_size if (i + 1) * chunk_size < num_rows else num_rows

        chunk_df = dataframe.iloc[start_idx:end_idx]

        chunk_df.to_sql(schema= schema_name, name = table_name, con =conn, index = False, if_exists = 'append', chunksize = 5000, method = 'multi')

    print("Data loaded successfully.")
    

In [0]:
%%time
ld = pd.read_sql_query(f'''
                    select
                    ml.id as loan_id,
                    ml.account_no,
                    ml.external_id,
                    ml.client_id,
                    ml.group_id,
                    replace(ifnull(mc.display_name, mg.display_name), 'null', '') as client_name,
                    case when isnull(ml.group_id) then 'No' else 'Yes' end as 'is_group',
                    
                    ml.glim_id,
                    ml.product_id,
                    mpl.name as product_name,
                    ml.fund_id,
                    
                    ml.loan_officer_id,
                    i.officer_name as loan_officername,
                    
                    ml.loanpurpose_cv_id,
                    g.code_value as loan_purpose,
                    
                    ml.loan_status_id,
                    a.enum_message_property as loan_status,
                    
                    ml.loan_type_enum,
                    lt.enum_message_property as loan_type,
                    
                    ml.currency_code,
                    ml.currency_digits,
                    ml.currency_multiplesof,
                    ml.principal_amount_proposed,
                    ml.principal_amount,
                    ml.approved_principal,
                    ml.net_disbursal_amount,
                    ml.arrearstolerance_amount,
                    ml.is_floating_interest_rate,
                    ml.interest_rate_differential,
                    ml.nominal_interest_rate_per_period,
                    
                    ml.interest_period_frequency_enum,
                    b.enum_message_property as interest_period_frequency,
                    
                    ml.annual_nominal_interest_rate,
                    
                    ml.interest_method_enum,
                    c.enum_message_property as interest_method,
                    
                    ml.interest_calculated_in_period_enum,
                    d.enum_message_property as interest_calculation_period,
                    
                    ml.allow_partial_period_interest_calcualtion,
                    ml.term_frequency,
                    
                    ml.term_period_frequency_enum,
                    f.enum_message_property as term_period_frequency,
                    
                    ml.repay_every,
                    
                    ml.repayment_period_frequency_enum,
                    ab.enum_message_property as repayment_period_frequency,
                    
                    ml.number_of_repayments,
                    ml.grace_on_principal_periods,
                    ml.recurring_moratorium_principal_periods,
                     ml.grace_on_interest_periods,
                     ml.grace_interest_free_periods,
                     
                     ml.amortization_method_enum,
                     am.enum_message_property as amortization_method,
                     
                     ml.submittedon_date,
                     
                     ml.submittedon_userid,
                     j.officer_name as submitted_by_officername,
                     
                     ml.approvedon_date,
                     ml.approvedon_userid,
                     k.officer_name as approved_by_officername,
                     
                     ml.expected_disbursedon_date,
                     ml.expected_firstrepaymenton_date,
                     ml.interest_calculated_from_date,
                     ml.disbursedon_date,
                     ml.disbursedon_userid,
                     l.officer_name as disbursed_by_officername,
                     
                     ml.expected_maturedon_date,
                     ml.maturedon_date,
                     ml.closedon_date,
                     
                     ml.closedon_userid,
                     m.officer_name as closed_by_officername,
                     
                     ml.total_charges_due_at_disbursement_derived,
                     ml.principal_disbursed_derived,
                     ml.principal_repaid_derived,
                     ml.principal_writtenoff_derived,
                     ml.principal_outstanding_derived,
                     ml.interest_charged_derived,
                     ml.interest_repaid_derived,
                     ml.interest_waived_derived,
                     ml.interest_writtenoff_derived,
                     ml.interest_outstanding_derived,
                     ml.fee_charges_charged_derived,
                     ml.fee_charges_repaid_derived,
                     ml.fee_charges_waived_derived,
                     ml.fee_charges_writtenoff_derived,
                     ml.fee_charges_outstanding_derived,
                     ml.penalty_charges_charged_derived,
                     ml.penalty_charges_repaid_derived,
                     ml.penalty_charges_waived_derived,
                     ml.penalty_charges_writtenoff_derived,
                     ml.penalty_charges_outstanding_derived,
                     ml.total_expected_repayment_derived,
                     ml.total_repayment_derived,
                     ml.total_expected_costofloan_derived,
                     ml.total_costofloan_derived,
                    ml.total_waived_derived,
                     ml.total_writtenoff_derived,
                     ml.total_outstanding_derived,
                     ml.total_overpaid_derived,
                     ml.rejectedon_date,
                     
                     ml.rejectedon_userid,
                     n.officer_name as rejected_by_officername,
                     
                     ml.rescheduledon_date,
                     
                     ml.rescheduledon_userid,
                     o.officer_name as rescheduled_by_officername,
                     
                     ml.withdrawnon_date,
                     
                     ml.withdrawnon_userid,
                     p.officer_name as withdrawn_by_officername,
                     
                     ml.writtenoffon_date,
                     
                     
                     ml.loan_transaction_strategy_id,
                     e.enum_message_property as loan_transaction_strategy,
                     
                     ml.sync_disbursement_with_meeting,
                     ml.loan_counter,
                     ml.loan_product_counter,
                     ml.fixed_emi_amount,
                     ml.max_outstanding_loan_balance,
                     ml.grace_on_arrears_ageing,
                     ml.is_npa,
                     ml.total_recovered_derived,
                     ml.accrued_till,
                     ml.interest_recalcualated_on,
                     ml.days_in_month_enum,
                     ml.days_in_year_enum,
                     ml.interest_recalculation_enabled,
                     ml.guarantee_amount_derived,
                     ml.create_standing_instruction_at_disbursement,
                     ml.version,
                     ml.writeoff_reason_cv_id,
                     h.code_value as writeoff_reason,
                     
                     ml.loan_sub_status_id,
                     ml.is_topup,
                     ml.is_equal_amortization,
                     ml.fixed_principal_percentage_per_installment,
                     now() as load_date

                    from m_loan ml
                    left join m_client mc on mc.id = ml.client_id
                    left join m_group mg on mg.id = ml.group_id
                    left join m_product_loan mpl  on mpl.id = ml.product_id

                    left join (select enum_id, enum_message_property from r_enum_value where enum_name = 'loan_status_id') a on a.enum_id = ml.loan_status_id
                    
                    left join (select enum_id, enum_message_property from r_enum_value where enum_name = 'loan_type_enum') lt on lt.enum_id = ml.loan_type_enum
                    
                    left join (select enum_id, enum_message_property from r_enum_value where enum_name = 'interest_period_frequency_enum') b on b.enum_id = ml.interest_period_frequency_enum
                    
                    left join (select enum_id, enum_message_property from r_enum_value where enum_name = 'interest_method_enum') c on c.enum_id = ml.interest_method_enum
                    
                    left join (select enum_id, enum_message_property from r_enum_value where enum_name = 'interest_calculated_in_period_enum') d on d.enum_id = ml.interest_calculated_in_period_enum                    
                    
                    left join (select enum_id, enum_message_property from r_enum_value where enum_name = 'loan_transaction_strategy_id') e on e.enum_id = ml.loan_transaction_strategy_id 
                    
                    left join (select enum_id, enum_message_property from r_enum_value where enum_name = 'repayment_period_frequency_enum') ab on ab.enum_id = ml.repayment_period_frequency_enum 
                    
                    left join (select enum_id, enum_message_property from r_enum_value where enum_name = 'amortization_method_enum') am on am.enum_id = ml.amortization_method_enum 
                    
                    left join (select enum_id, enum_message_property from r_enum_value where enum_name = 'term_period_frequency_enum') f on f.enum_id = ml.term_period_frequency_enum 
                    
                    left join (select code_id, code_value from m_code_value where code_id = 3) g on g.code_id = ml.loanpurpose_cv_id
                    
                    left join (select code_id, code_value from m_code_value where code_id = 26) h on h.code_id = ml.writeoff_reason_cv_id
                    
                    left join (select id, concat(firstname, ' ', lastname) as officer_name from m_appuser) i on i.id = ml.loan_officer_id
                    
                    left join (select id, concat(firstname, ' ', lastname) as officer_name from m_appuser) j on j.id = ml.submittedon_userid
                    
                    left join (select id, concat(firstname, ' ', lastname) as officer_name from m_appuser) k on k.id = ml.approvedon_userid
                    
                    left join (select id, concat(firstname, ' ', lastname) as officer_name from m_appuser) l on l.id = ml.disbursedon_userid
                    
                    left join (select id, concat(firstname, ' ', lastname) as officer_name from m_appuser) m on m.id = ml.closedon_userid
                    
                    left join (select id, concat(firstname, ' ', lastname) as officer_name from m_appuser) n on n.id = ml.rejectedon_userid
                    
                    left join (select id, concat(firstname, ' ', lastname) as officer_name from m_appuser) o on o.id = ml.rescheduledon_userid
                    
                    left join (select id, concat(firstname, ' ', lastname) as officer_name from m_appuser) p on p.id = ml.withdrawnon_userid
                            
                    
                    ''', Mifosdb)

ld.head()

In [0]:
ld.shape

In [0]:
%%time
load_table('public','dwh_loan_details', ld )
#ld.to_sql('dwh_loan_details', conn, index = False, if_exists = 'replace', chunksize = 10000, method = 'multi')

## Loan repayment schedule etl

In [0]:
%%time
# loan repayment schedule table

mlrs = pd.read_sql(f'''
                select
                *
                from m_loan_repayment_schedule mlrs
                ''', Mifosdb)

mlrs = pd.merge(mlrs, ld[['loan_id', 'loan_status_id', 'disbursedon_date', 'maturedon_date', 'number_of_repayments', 
                          'principal_disbursed_derived', 'total_outstanding_derived', 'interest_charged_derived','load_date']],
                on = 'loan_id', how = 'left')
mlrs.head()

In [0]:
mlrs.shape

In [0]:
%%time 
load_table('public','dwh_loan_repayment_schedule', mlrs )
#mlrs.to_sql("dwh_loan_repayment_schedule", conn, index = False, if_exists = 'replace', chunksize = 10000, method = 'multi')

### Daily expected repayment aggregate etl 2022 to Date - 1 (t-1)

In [0]:
%%time
dexprp = pd.read_sql(f'''
                select
                duedate as date,
                sum(principal_amount + interest_amount) as total_expected_repayment
                from dwh_loan_repayment_schedule
                where left(duedate, 10) >= '2022-01-01' -- and left(duedate,10) < current_date
                group by 1
                order by 1
                ''', conn)
dexprp['date'] = dexprp['date'].astype(str) 

In [0]:
actrp = pd.read_sql(f'''
                select
                left(transaction_date, 10) as date,
                sum(amount) as total_actual_repayment,
                getdate() as load_date
                from dwh_loan_transactions
                where transaction_type_enum = 2
                and left(transaction_date, 10) >= '2022-01-01' -- and left(transaction_date, 10) < current_date
                group by 1
                order by 1
                ''', conn)

actrp['date'] = actrp['date'].astype(str)

In [0]:
loan_rpmt = pd.merge(dexprp, actrp, on = 'date', how = 'left')
loan_rpmt.head()

In [0]:
%%time
load_table('public','dwh_daily_expected_vs_actual_repayment', loan_rpmt)
#loan_rpmt.to_sql("dwh_daily_expected_vs_actual_repayment", conn, index = False, if_exists = 'replace', chunksize = 10000, method = 'multi')

## Staff Loan performance 

In [0]:
## ref codes 

rfby = pd.read_sql(f'''
                select
                mc.id as client_id,
                ifnull(mc.referred_by_id, 0) as referred_by_id
                from m_client mc
                where mc.id in ({','.join([str(x) for x in ld['client_id'].tolist()])})
                ''', Mifosdb)


rfcd = pd.read_sql(f'''
            select
            mc.id as referred_by_id,
            mc.referral_id as staff_code
            from m_client mc
            where mc.id in ({','.join([str(x) for x in rfby['referred_by_id'].tolist()])})
            ''', Mifosdb)


rf = pd.merge(rfby, rfcd, on = 'referred_by_id', how = 'inner')

rf.head()

In [0]:
#rfby['referred_by_id'].dtype

In [0]:
final = pd.merge(ld, rf, on = 'client_id', how = 'inner')
final.head()

In [0]:
%%time
load_table('public','dwh_staff_loan_performance', final)
#final.to_sql("dwh_staff_loan_performance", conn, index = False, if_exists = 'replace', chunksize = 10000, method = 'multi')